<a href="https://colab.research.google.com/github/harnalashok/CatEncodersFamily/blob/main/network_connect_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 23rd May, 2023
# For Updated code see:
#Folders: C:\Users\Ashok\OneDrive\Documents\talkingdata\languagemodeling
#         C:\Users\Ashok\OneDrive\Documents\skipgrams

#Relevant files:
#https://github.com/harnalashok/deeplearning-sequences/blob/main/skipgram_generation.ipynb

In [1]:
!pip install cdlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 120.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 123.6 MB/s eta 0:00:00
  Created wheel for python-igraph: filename=python_igraph-0.10.4-py3-none-any.whl size=9073 sha256=5cc7ae8093778a08b04751db4d439c999ebd67a83992663b1cc3fe4ede652f03
  Stored in directory: /root/.cache/pip/wheels/49/27/15/dcf90953e3e3322e6f3e447514b20cf39b53c6518cb4a7bace
Successfully built python-igraph


In [4]:
!pip install catencfamily

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [10]:
%reset -f

# 1.0 Call libraries
import pandas as pd
import numpy as np
import networkx as nx

# 1.01
import joblib
from collections import defaultdict

# 1.02
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# 1.08 Misc
import os,gc , time,  glob
from os.path import exists
import re
import ast
# 1.04
from catencfamily.encoders import CatEncodersFamily
from catencfamily import utils


Note: to be able to use all crisp methods, you need to install some additional packages:  {'bayanpy', 'leidenalg', 'graph_tool', 'wurlitzer', 'infomap'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap', 'leidenalg'}


In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [15]:
# Paths:
modulePath = "/gdrive/MyDrive/Colab_data_files/talkingData/03112022/"
dataPath = "/gdrive/MyDrive/Colab_data_files/skipgram/"
modelsPath = "/gdrive/MyDrive/Colab_data_files/skipgram/modelspath/"
pathToStoreProgress = "/gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/"
mapdictPath = "/gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/"

In [11]:
# Cleaning up
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/modelspath/
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/

In [12]:
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/vectors.tsv
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/metadata.tsv

In [13]:

!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/modelspath/
!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/
!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/

In [16]:
# 8.0 Read back pkl file and dictionary:
os.chdir(dataPath)
#train = pd.read_pickle("seq.pkl")
train = pd.read_pickle("skipgrams.pkl")
train.head()

,a,b,c,d,e,f,g,h,i,j
0,761,493,4,2156,42,242,35,986,70,4
1,493,4,2156,42,242,35,986,70,4,29
2,4,2156,42,242,35,986,70,4,29,5
3,2156,42,242,35,986,70,4,29,5,2
4,42,242,35,986,70,4,29,5,2,987


In [ ]:
# Read dictionary
os.chdir(dataPath)
filehandler = open("word_index.txt", 'r')
word2index = filehandler.read()
word2index

In [19]:
org_columns = train.columns
cat_cols = ['a', 'b','c', 'd','e', 'f', 'g', 'h', 'i', 'j']
interactingCatCols = cat_cols

In [20]:
# 5.0 Instantiate CatEncodersFamily class:

ct = CatEncodersFamily(
                       cMeasures=[1,1,1,1,None,1,1],
                       k = 40
                       )

# 5.1 Fit it using only train_binned dataset.
#     While fitting, 'target' is not needed:


ct.fit(train, cat_cols, interactingCatCols)

Record set no is 1  
Set of records is:  [('a', 'b', [1, 1, 1, 1, None, 1, 1]), ('a', 'c', [1, 1, 1, 1, None, 1, 1]), ('a', 'd', [1, 1, 1, 1, None, 1, 1]), ('a', 'e', [1, 1, 1, 1, None, 1, 1]), ('a', 'f', [1, 1, 1, 1, None, 1, 1]), ('a', 'g', [1, 1, 1, 1, None, 1, 1]), ('a', 'h', [1, 1, 1, 1, None, 1, 1]), ('a', 'i', [1, 1, 1, 1, None, 1, 1]), ('a', 'j', [1, 1, 1, 1, None, 1, 1]), ('a', 'e_p_h', [1, 1, 1, 1, None, 1, 1]), ('a', 'c_p_f', [1, 1, 1, 1, None, 1, 1]), ('a', 'd_p_h', [1, 1, 1, 1, None, 1, 1]), ('a', 'a_p_f', [1, 1, 1, 1, None, 1, 1]), ('a', 'f_p_h', [1, 1, 1, 1, None, 1, 1]), ('a', 'c_p_h', [1, 1, 1, 1, None, 1, 1]), ('a', 'c_p_g', [1, 1, 1, 1, None, 1, 1]), ('a', 'a_p_j', [1, 1, 1, 1, None, 1, 1]), ('a', 'b_p_d', [1, 1, 1, 1, None, 1, 1]), ('a', 'f_p_i', [1, 1, 1, 1, None, 1, 1]), ('a', 'b_p_i', [1, 1, 1, 1, None, 1, 1]), ('a', 'd_p_e', [1, 1, 1, 1, None, 1, 1]), ('a', 'd_p_f', [1, 1, 1, 1, None, 1, 1]), ('a', 'a_p_d', [1, 1, 1, 1, None, 1, 1]), ('a', 'c_p_i', [1, 1, 1, 1, 

KeyboardInterrupt: ignored

In [ ]:
def readStoredModels(pathToReadFrom, fileList):
  model_dfs = []
  os.chdir(pathToReadFrom)
  for file in fileList:
    df = pd.read_pickle(file)
    model_dfs.append(df)
  return model_dfs


In [ ]:
os.chdir(modelsPath)
files = os.listdir()
file_list = [file for file in files if ("deg_a_ck" in file) or ("eig_a_ck" in file) or ("pr_a_ck" in file) or ("nf_a_ck" in file)]


In [ ]:
files

In [ ]:
#models = readStoredModels(modelsPath, file_list)
models = readStoredModels(modelsPath, files)

In [ ]:
# Merge models
for i, model in enumerate(models):
  if i == 0:
    df = model
  else:
    df = pd.merge(df, model, left_index=True, right_index=True)

In [ ]:
df

,deg_a_ck_b,eig_a_ck_b,pr_a_ck_b,nf_a_ck_b,deg_a_ck_c,eig_a_ck_c,pr_a_ck_c,nf_a_ck_c,deg_a_ck_d,eig_a_ck_d,...,pr_a_ck_b_p_d,nf_a_ck_b_p_d,deg_a_ck_a_p_c,eig_a_ck_a_p_c,pr_a_ck_a_p_c,nf_a_ck_a_p_c,deg_a_ck_a_p_d,eig_a_ck_a_p_d,pr_a_ck_a_p_d,nf_a_ck_a_p_d
a,,,,,,,,,,,,,,,,,,,,,
1.0,0.248392,1.299868e-01,0.006540,-0.015572,0.281296,0.109392,0.006091,0.057763,0.306531,0.113723,...,0.006292,-0.359157,0.0,-0.005525,0.000247,0.015727,0.0,0.010881,0.000247,0.015727
2.0,0.198911,1.178593e-01,0.005402,0.004240,0.200148,0.076445,0.004680,0.022102,0.256061,0.085578,...,0.002304,0.002183,0.0,-0.015954,0.000247,0.015727,0.0,-0.025699,0.000247,0.015727
3.0,0.161554,1.015974e-01,0.004420,-0.000270,0.190252,0.090422,0.004063,-0.024636,0.206333,0.097460,...,0.003428,-0.044276,0.0,0.015868,0.000247,0.015727,0.0,-0.011920,0.000247,0.015727
4.0,0.163038,1.010449e-01,0.004065,0.004713,0.157843,0.069505,0.003070,-0.051084,0.184810,0.077516,...,0.002488,-0.054258,0.0,0.018441,0.000247,0.015727,0.0,0.013114,0.000247,0.015727
5.0,0.161554,1.060454e-01,0.004075,-0.007628,0.196932,0.097576,0.003854,0.021942,0.232558,0.102975,...,0.005289,-0.081413,0.0,0.016620,0.000247,0.015727,0.0,0.003003,0.000247,0.015727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4039.0,0.000000,8.508111e-19,0.000043,0.001275,0.001484,0.000340,0.000079,-0.012828,0.001484,0.000878,...,0.000136,0.007987,0.0,-0.002388,0.000247,0.015727,0.0,-0.001351,0.000247,0.015727
4040.0,0.001484,4.553849e-04,0.000136,-0.012791,0.001484,0.000692,0.000090,0.015095,0.000495,0.000313,...,0.000136,0.007987,0.0,0.022754,0.000247,0.015727,0.0,0.017860,0.000247,0.015727
4041.0,0.000000,-6.535554e-19,0.000043,0.001275,0.000990,0.000440,0.000067,0.002836,0.003958,0.001139,...,0.000136,0.007987,0.0,0.003664,0.000247,0.015727,0.0,-0.004688,0.000247,0.015727


In [ ]:
# Remove constant columns
df = df.loc[:, (df != df.iloc[0]).any()]
df.head()

,deg_a_ck_b,eig_a_ck_b,pr_a_ck_b,nf_a_ck_b,deg_a_ck_c,eig_a_ck_c,pr_a_ck_c,nf_a_ck_c,deg_a_ck_d,eig_a_ck_d,...,deg_a_ck_b_p_c,eig_a_ck_b_p_c,pr_a_ck_b_p_c,nf_a_ck_b_p_c,deg_a_ck_b_p_d,eig_a_ck_b_p_d,pr_a_ck_b_p_d,nf_a_ck_b_p_d,eig_a_ck_a_p_c,eig_a_ck_a_p_d
a,,,,,,,,,,,,,,,,,,,,,
1.0,0.248392,0.129987,0.006540,-0.015572,0.281296,0.109392,0.006091,0.057763,0.306531,0.113723,...,0.024245,0.073086,0.006856,-0.133703,0.009154,0.408504,0.006292,-0.359157,-0.005525,0.010881
2.0,0.198911,0.117859,0.005402,0.004240,0.200148,0.076445,0.004680,0.022102,0.256061,0.085578,...,0.007669,0.002746,0.004189,0.073910,0.001979,0.014685,0.002304,0.002183,-0.015954,-0.025699
3.0,0.161554,0.101597,0.004420,-0.000270,0.190252,0.090422,0.004063,-0.024636,0.206333,0.097460,...,0.008164,0.021135,0.003349,-0.011033,0.004206,0.092681,0.003428,-0.044276,0.015868,-0.011920
4.0,0.163038,0.101045,0.004065,0.004713,0.157843,0.069505,0.003070,-0.051084,0.184810,0.077516,...,0.011875,0.019998,0.003539,0.036896,0.003216,0.100516,0.002488,-0.054258,0.018441,0.013114
5.0,0.161554,0.106045,0.004075,-0.007628,0.196932,0.097576,0.003854,0.021942,0.232558,0.102975,...,0.010143,0.026415,0.003218,-0.085058,0.006927,0.150771,0.005289,-0.081413,0.016620,0.003003


In [ ]:
# Save dataframe:
os.chdir(dataPath)
df.to_pickle("word2vec.pkl")


In [ ]:
# Load word2vec
os.chdir(dataPath)
df_word= pd.read_pickle("word2vec.pkl")
df_word.head()
df_word.tail()

,deg_a_ck_b,eig_a_ck_b,pr_a_ck_b,nf_a_ck_b,deg_a_ck_c,eig_a_ck_c,pr_a_ck_c,nf_a_ck_c,deg_a_ck_d,eig_a_ck_d,...,deg_a_ck_b_p_c,eig_a_ck_b_p_c,pr_a_ck_b_p_c,nf_a_ck_b_p_c,deg_a_ck_b_p_d,eig_a_ck_b_p_d,pr_a_ck_b_p_d,nf_a_ck_b_p_d,eig_a_ck_a_p_c,eig_a_ck_a_p_d
a,,,,,,,,,,,,,,,,,,,,,
1.0,0.248392,0.129987,0.006540,-0.015572,0.281296,0.109392,0.006091,0.057763,0.306531,0.113723,...,0.024245,0.073086,0.006856,-0.133703,0.009154,0.408504,0.006292,-0.359157,-0.005525,0.010881
2.0,0.198911,0.117859,0.005402,0.004240,0.200148,0.076445,0.004680,0.022102,0.256061,0.085578,...,0.007669,0.002746,0.004189,0.073910,0.001979,0.014685,0.002304,0.002183,-0.015954,-0.025699
3.0,0.161554,0.101597,0.004420,-0.000270,0.190252,0.090422,0.004063,-0.024636,0.206333,0.097460,...,0.008164,0.021135,0.003349,-0.011033,0.004206,0.092681,0.003428,-0.044276,0.015868,-0.011920
4.0,0.163038,0.101045,0.004065,0.004713,0.157843,0.069505,0.003070,-0.051084,0.184810,0.077516,...,0.011875,0.019998,0.003539,0.036896,0.003216,0.100516,0.002488,-0.054258,0.018441,0.013114
5.0,0.161554,0.106045,0.004075,-0.007628,0.196932,0.097576,0.003854,0.021942,0.232558,0.102975,...,0.010143,0.026415,0.003218,-0.085058,0.006927,0.150771,0.005289,-0.081413,0.016620,0.003003


,deg_a_ck_b,eig_a_ck_b,pr_a_ck_b,nf_a_ck_b,deg_a_ck_c,eig_a_ck_c,pr_a_ck_c,nf_a_ck_c,deg_a_ck_d,eig_a_ck_d,...,deg_a_ck_b_p_c,eig_a_ck_b_p_c,pr_a_ck_b_p_c,nf_a_ck_b_p_c,deg_a_ck_b_p_d,eig_a_ck_b_p_d,pr_a_ck_b_p_d,nf_a_ck_b_p_d,eig_a_ck_a_p_c,eig_a_ck_a_p_d
a,,,,,,,,,,,,,,,,,,,,,
4039.0,0.000000,8.508111e-19,0.000043,0.001275,0.001484,0.000340,0.000079,-0.012828,0.001484,0.000878,...,0.0,-2.463631e-18,0.000103,0.008186,0.0,-2.009798e-18,0.000136,0.007987,-0.002388,-0.001351
4040.0,0.001484,4.553849e-04,0.000136,-0.012791,0.001484,0.000692,0.000090,0.015095,0.000495,0.000313,...,0.0,2.009796e-18,0.000103,0.008186,0.0,1.422856e-18,0.000136,0.007987,0.022754,0.017860
4041.0,0.000000,-6.535554e-19,0.000043,0.001275,0.000990,0.000440,0.000067,0.002836,0.003958,0.001139,...,0.0,1.429341e-18,0.000103,0.008186,0.0,-1.461007e-18,0.000136,0.007987,0.003664,-0.004688
4042.0,0.000990,4.485260e-04,0.000081,-0.000614,0.003958,0.002579,0.000116,-0.009662,0.021524,0.012136,...,0.0,2.565369e-18,0.000103,0.008186,0.0,2.568571e-18,0.000136,0.007987,0.012466,0.018826
4043.0,0.000495,8.675064e-04,0.000056,-0.000166,0.032905,0.028461,0.000508,0.002912,0.000495,0.000007,...,0.0,2.247635e-18,0.000103,0.008186,0.0,1.682161e-18,0.000136,0.007987,-0.010286,0.024826


In [ ]:
# Standardize vectors
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df_word_ss = ss.fit_transform(df_word)

In [ ]:
df_word_ss = pd.DataFrame(df_word_ss, columns = df_word.columns)
df_word_ss

In [ ]:
os.chdir(dataPath)
df_word.to_csv('vectors.tsv', sep="\t", index = False, header = False)
df_word_ss.to_csv('vectors_ss.tsv', sep="\t", index = False, header = False)

In [ ]:
metadata = ast.literal_eval(re.search('({.+})', word2index).group(0))

In [ ]:
str_list = list(metadata.keys())
str_list

In [ ]:
import sys
orig_stdout = sys.stdout
f = open('metadata.tsv', 'w')
sys.stdout = f

print(' \n'.join(str_list))

sys.stdout = orig_stdout
f.close()
sys.stdout = orig_stdout

In [ ]:
######## I am done ################3